heart failure from https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data

In [1]:
import numpy as np
import pandas as pd
import ripser
from sklearn import metrics
from persim.persistent_entropy import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

In [100]:
df = pd.read_csv("../Data/heart_failure_clinical_records_dataset.csv")
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [114]:
columns_to_scale = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 'serum_sodium']
scaler = StandardScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

In [115]:
X = df.drop(columns=['DEATH_EVENT', 'time'])
y = df['DEATH_EVENT']
print(X.shape)

(299, 11)


In [116]:
entropy_feature = []
new_X = X[columns_to_scale]
for i in range(X.shape[0]):
    # data_remove = X.drop(index=i)
    data_remove = new_X.drop(index=i)
    dgm = ripser.ripser(data_remove)['dgms'][0]
    p_entropy = persistent_entropy(dgm)
    entropy_feature.append(p_entropy)

In [117]:
X['topo_feature'] = [x.tolist()[0] if isinstance(x, np.ndarray) else x for x in entropy_feature]
X.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,topo_feature
0,1.192945,0,0.000166,0,-1.530560,1,1.681648e-02,0.490057,-1.504036,1,0,5.557136
1,-0.491279,0,7.514640,0,-0.007077,0,7.535660e-09,-0.284552,-0.141976,1,0,5.558072
2,0.350833,0,-0.449939,0,-1.530560,0,-1.038073e+00,-0.090900,-1.731046,1,1,5.557460
3,-0.912335,1,-0.486071,0,-1.530560,0,-5.464741e-01,0.490057,0.085034,1,0,5.557206
4,0.350833,1,-0.435486,1,-1.530560,0,6.517986e-01,1.264666,-4.682176,0,0,5.556417


In [118]:
X['topo_feature'] = scaler.fit_transform(X[['topo_feature']])
# columns_to_scale.append('topo_feature')
# X[columns_to_scale] = scaler.fit_transform(X[columns_to_scale])

X.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,topo_feature
0,1.192945,0,0.000166,0,-1.530560,1,1.681648e-02,0.490057,-1.504036,1,0,-0.316537
1,-0.491279,0,7.514640,0,-0.007077,0,7.535660e-09,-0.284552,-0.141976,1,0,0.484281
2,0.350833,0,-0.449939,0,-1.530560,0,-1.038073e+00,-0.090900,-1.731046,1,1,-0.039139
3,-0.912335,1,-0.486071,0,-1.530560,0,-5.464741e-01,0.490057,0.085034,1,0,-0.256522
4,0.350833,1,-0.435486,1,-1.530560,0,6.517986e-01,1.264666,-4.682176,0,0,-0.931393


In [124]:
X_final = X.iloc[:, :11].to_numpy()
print(X_final.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.5, random_state=42)

log_reg = LogisticRegression(max_iter = 1000)
log_reg.fit(X_train, y_train)

training_prediction = log_reg.predict(X_train)
test_prediction = log_reg.predict(X_test)

print("Precision, Recall, Confusion matrix, in training\n")
print(metrics.confusion_matrix(y_train, training_prediction))
print(metrics.accuracy_score(y_train, training_prediction))

(299, 11)
Precision, Recall, Confusion matrix, in training

[[105   3]
 [ 25  16]]
0.8120805369127517


In [123]:
X_final = X.to_numpy()
print(X_final.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.5, random_state=42)

log_reg = LogisticRegression(max_iter = 1000)
log_reg.fit(X_train, y_train)

training_prediction = log_reg.predict(X_train)
test_prediction = log_reg.predict(X_test)

print("Precision, Recall, Confusion matrix, in training\n")
print(metrics.confusion_matrix(y_train, training_prediction))
print(metrics.accuracy_score(y_train, training_prediction))

(299, 12)
Precision, Recall, Confusion matrix, in training

[[105   3]
 [ 24  17]]
0.8187919463087249
